In [112]:
import random
from collections import Counter
import itertools
import numpy as np

In [2]:
def hammingDistance(p, q):
    count = 0
    for i in range(len(p)):
        if p[i] != q[i]:
            count += 1
    return count

In [3]:
def profile_probability(consensus, profile):
    prob = 1
    for i in range(len(consensus)):
        prob *= profile[consensus[i]][i]
    return prob

In [4]:
def profile_to_consensus(profile):
    
    nu_list = ['A','C','G','T']
    nu_candidates = []
    consensus_list = []
    for i in range(len(profile['A'])):
        current_col = [profile[x][i] for x in nu_list]
        max_freq = max(enumerate(current_col),key=lambda x: x[1])[1]
        current_candidates = []
        for j in range(len(current_col)):
            if current_col[j] == max_freq:
                current_candidates.append(nu_list[j])
        nu_candidates.append(current_candidates)
    #print(nu_candidates)
    for x in list(itertools.product(*nu_candidates)):
        consensus_list.append(''.join(y for y in x))
    return consensus_list

In [5]:
def create_profile(motifs):
    col_len = len(motifs)
    A_list = []
    C_list = []
    G_list = []
    T_list = []
    for i in range(len(motifs[0])):
        col_nucleotides = [each_motif[i] for each_motif in motifs]
        c = Counter(col_nucleotides)
        A_list.append(c['A']/col_len)
        C_list.append(c['C']/col_len)
        G_list.append(c['G']/col_len)
        T_list.append(c['T']/col_len)
    profile = {'A':A_list, 'C':C_list, 'G':G_list, 'T':T_list}
    return profile

In [6]:
def create_profile_laplacian(motifs):
    col_len = len(motifs)
    A_list = []
    C_list = []
    G_list = []
    T_list = []
    for i in range(len(motifs[0])):
        col_nucleotides = [each_motif[i] for each_motif in motifs]
        c = Counter(col_nucleotides)
        A_list.append((c['A']+1)/(2*col_len))
        C_list.append((c['C']+1)/(2*col_len))
        G_list.append((c['G']+1)/(2*col_len))
        T_list.append((c['T']+1)/(2*col_len))
    profile = {'A':A_list, 'C':C_list, 'G':G_list, 'T':T_list}
    return profile      

In [7]:
def profile_most_probable_kmer(dna, k, profile):
    prob = 0
    most_probable_kmer = ''
    for i in range(len(dna)-k+1):
        current_kmer = dna[i:i+k]
        if profile_probability(current_kmer, profile) > prob:
            prob = profile_probability(current_kmer, profile)
            most_probable_kmer = current_kmer 
            
    if prob == 0:
        most_probable_kmer = dna[:k]
    return most_probable_kmer

In [52]:
def score(motifs):
    profile = create_profile_laplacian(motifs)
    consensus = profile_to_consensus(profile)[0]
    total_score = 0
    for i in motifs:
        total_score += hammingDistance(consensus, i)
    return total_score
    

In [57]:
def randomizedMotifSearch(dna, k, t):
    random_indexes = [random.randint(0, len(dna[0])-k) for _ in range(t)]
    #print(random_indexes)
    motifs = [dna[i][random_indexes[i]:random_indexes[i]+k] for i in range(len(random_indexes))]
    bestMotifs = motifs
    #print(bestMotifs)
    while True:
        profile = create_profile_laplacian(motifs)
        motifs = [profile_most_probable_kmer(x, k, profile) for x in dna]
        #print(motifs)
        if score(motifs) < score(bestMotifs):
            bestMotifs = motifs
        else:
            return bestMotifs

In [174]:
sample_dna = ['CGCCCCTCTCGGGGGTGTTCAGTAAACGGCCA','GGGCGAGGTATGTGTAAGTGCCAAGGTGCCAG','TAGTACCGAGACCGAAAGAAGTATACAGGCGT','TAGATCAAGTTTCAGGTGCACGTCGGTGAACC','AATCCACCAGCTCCACGTGCAATGTTGGCCTA']

In [58]:
randomizedMotifSearch(sample_dna, 8, 5)

['GGTGTTCA', 'AAGTGCCA', 'GAGACCGA', 'AAGTTTCA', 'CAGCTCCA']

In [59]:
score(randomizedMotifSearch(sample_dna, 8, 5))

13

In [97]:
score_list = []
bestMotifs_list = []
result_list = []
for i in range(1000):
    bestMotifs = randomizedMotifSearch(real_data[1:-1], 15, 20)
    score_list.append(score(bestMotifs))
    bestMotifs_list.append(bestMotifs)

min_score = min(score_list)
for i in range(len(score_list)):
    if score_list[i] == min_score:
        result_list.append(tuple(bestMotifs_list[i]))

print(Counter(result_list).most_common()[0][0])




    

('TTGTTAGGCACACAA', 'TGGCTTACCAAAAAT', 'ACGTCAACCAAAAAG', 'TGGTTTTCCAAAAAT', 'TGGTCAACACGAAAT', 'TGGTCCTACAAAAAT', 'TGGTCAACCCCCAAT', 'TGGTCAGAAAAAAAT', 'TGTCGAACCAAAAAT', 'TGGTCAAGGGAAAAT', 'TGGTCAACCATTGAT', 'TGGTAGCCCAAAAAT', 'TGGTCATATAAAAAT', 'AGGTCAACCAAAACA', 'TGGTCAACCAACTCT', 'TGGGATACCAAAAAT', 'AATTCAACCAAAAAT', 'TGGTCGTACAAAAAT', 'TAAACAACCAAAAAT', 'TGGTCAACCAAATCC')


In [99]:
for i in bb:
    print(i)

TTGTTAGGCACACAA
TGGCTTACCAAAAAT
ACGTCAACCAAAAAG
TGGTTTTCCAAAAAT
TGGTCAACACGAAAT
TGGTCCTACAAAAAT
TGGTCAACCCCCAAT
TGGTCAGAAAAAAAT
TGTCGAACCAAAAAT
TGGTCAAGGGAAAAT
TGGTCAACCATTGAT
TGGTAGCCCAAAAAT
TGGTCATATAAAAAT
AGGTCAACCAAAACA
TGGTCAACCAACTCT
TGGGATACCAAAAAT
AATTCAACCAAAAAT
TGGTCGTACAAAAAT
TAAACAACCAAAAAT
TGGTCAACCAAATCC


In [170]:
with open('/home/jeongyeojin/dataset_163_4.txt') as f:
    data3 = f.read()
data3
real_data = data3.split('\n')
real_data

['15 20 2000',
 'TTCCGACATGTTGTGCTATCACGACGGAAGACCGCCAATATCTTGTTACTTGTCTTTTGACGCTAACCCTGTCACATTAAAGTGTAGACGATCCACTACAAACCCCCTCTATCGCGACGGGCTGTCAATCCACGATGATCATGTCACCAACTTTTAGATACAGAGTTCGCGAGCGATTTAAGTCGGTCCCGCACCGAAAATGAAAATGCCACTCGTGATGCGTGGAGCCGTGGATAGTACCAGGATTACCCTTAGCCGACATGATATCTTATGTTCTAGCGCCGCGATATCGCGCGTGCCACCGTTTTCCGACATGTTGTG',
 'CTATATCCGATCAAGAGCTCACGACGGAAGACCGCCAATATCTTGTTACTTGTCTTTTGACGCTAACCCTGTCACATTAAAGTGTAGACGATCCACTACAAACCCCCTCTATCGCGACGGGCTGTCAATCCACGATGATCATGTCACCAACTTTTAGATACAGAGTTCGCGAGCGATTTAAGTCGGTCCCGCACCGAAAATGAAAATGCCACTCGTGATGCGTGGAGCCGTGGATAGTACCAGGATTACCCTTAGCCGACATGATATCTTATGTTCTAGCGCCGCGATATCGCGCGTGCCACCGTTTTCCGACATGTTGTG',
 'AACGACCTGCGCGAACGCGATCGACGCGGGTTAATCACCGCCTGAACAAGCCGGGGATTCTGATAGACTACACCGCTGCCAGTACGGTCTTACTTCACGGCGCGGCAAGAATTTCTGATCCGCGGGTTACTCCTATGAGAAACGGCCATGTAAAAGTAGTCCACCCCTGTATTAGAACGAATGTTAAGTCTGCCGTCAATGGGGCCCGGGATTACGGCACTCGTGCCCCCCCGCGTTCACACGCTGACCACTCCCGCCTACGGTATAGAATTCTGAGCGGATATCATGGCGACCCGCTAACCCCTCGATCGTACCGTCTCA',
 'TTCCT

In [110]:
#probability that ten randomly selected 15-mers from the ten 600-nucleotide long strings in the Subtle Motif Problem capture at least one implanted 15-mer.
1-((585/586)**10)

0.01693439692911025

In [111]:
#probability that ten randomly selected 15-mers from ten 600-nucleotide long strings (as in the Subtle Motif Problem) capture at least two implanted 15-mers
1-((585/586)**10)-(10*(585**9)/586**10)

0.00012985670567623384

In [113]:
1-a

0.21029724648974146

In [114]:
def weighted_choice(weights):
    rnd = random.random()*sum(weights)
    for i, w in enumerate(weights):
        rnd -= w
        if rnd < 0:
            return i

In [159]:
def gibbsSampler(dna, k, t, N):
    random_indexes = [random.randint(0, len(dna[0])-k) for _ in range(t)]
    motifs = [dna[i][random_indexes[i]:random_indexes[i]+k] for i in range(len(random_indexes))]
    bestMotifs = motifs
    for j in range(N):
        i = random.randint(0, t-1)
        del motifs[i]
        profile = create_profile_laplacian(motifs)
        prob_list = []
        for x in range(len(dna[0])-k+1):
            prob_list.append(profile_probability(dna[i][x:x+k],profile))
        chosen_idx = weighted_choice(prob_list)
        ith_motif = dna[i][chosen_idx:chosen_idx+k]
        motifs.insert(i, ith_motif)
        best_score = score(bestMotifs)
        if score(motifs) < best_score:
            bestMotifs = motifs
    return bestMotifs
        
        

In [160]:
gibbs_dna = ['CGCCCCTCTCGGGGGTGTTCAGTAACCGGCCA','GGGCGAGGTATGTGTAAGTGCCAAGGTGCCAG','TAGTACCGAGACCGAAAGAAGTATACAGGCGT','TAGATCAAGTTTCAGGTGCACGTCGGTGAACC','AATCCACCAGCTCCACGTGCAATGTTGGCCTA']

In [171]:
score_list = []
bestMotifs_list = []
result_list = []
for i in range(20):
    bestMotifs = gibbsSampler(real_data[1:-1], 15, 20, 2000)
    score_list.append(score(bestMotifs))
    bestMotifs_list.append(bestMotifs)

min_score = min(score_list)
for i in range(len(score_list)):
    if score_list[i] == min_score:
        result_list.append(tuple(bestMotifs_list[i]))

print(Counter(result_list).most_common()[0][0])

('GATAGTACCAGGATT', 'TATCCGATCAAGAGC', 'TATAGAATTCTGAGC', 'TAATCAATCAAGAGC', 'ATAAGAATCAAGAGC', 'TATAGAATCAAGTCT', 'TATAGAAGACAGAGC', 'TATAGTTGCAAGAGC', 'TTACGAATCAAGAGC', 'TATAGAATCCGAAGC', 'TATACCGTCAAGAGC', 'CATAGAATCAAGACG', 'TATAGACCTAAGAGC', 'TATAGCTGCAAGAGC', 'TATAGAATCAGATGC', 'TATGACATCAAGAGC', 'TATAAGCTCAAGAGC', 'TATAGAATCAATTTC', 'TATAGAGGAAAGAGC', 'AGTAGAATCAAGAGA')
